In [1]:
# %pip install segment-geospatial groundingdino-py leafmap localtileserver

In [1]:
import os
import leafmap
from samgeo import split_raster
from samgeo.text_sam import LangSAM

In [2]:
def create_tiles_from_matches(matches, tile_out_dir="tiles", tile_size=(1000, 1000), overlap=0):
    """
    Create tiles for TIF images based on the matches dictionary.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and TIF image paths as values.
    tile_out_dir (str): Path to the output directory for tiles (default: "tiles").
    tile_size (tuple): Size of tiles for raster splitting (default: (1000, 1000)).
    overlap (int): Overlap between tiles (default: 0).
    """
    for als_file, dop_image in matches.items():
        # Extract file identifier from the ALS file path
        file_identifier = os.path.splitext(os.path.basename(als_file))[0]

        # Output directory for tiles of this TIF image
        tile_dir = os.path.join(tile_out_dir, file_identifier)

        # Ensure dop_image is a string, not a list
        if isinstance(dop_image, list):
            dop_image = dop_image[0]

        # Create tiles from the TIF image
        try:
            split_raster(filename=dop_image, out_dir=tile_dir, tile_size=tile_size, overlap=overlap)
            print(f"Tiles created for {file_identifier}. Saved in {tile_dir}.")
        except Exception as e:
            print(f"Error creating tiles for {file_identifier}: {e}")


In [3]:
def apply_sam_to_tiles_from_matches(matches, tile_base_dir="tiles", mask_out_dir="masks", text_prompt="tree", box_threshold=0.24, text_threshold=0.24):
    """
    Apply LangSAM to tiles based on the matches dictionary.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and DOP image paths as values.
    tile_base_dir (str): Path to the base directory containing tiles.
    mask_out_dir (str): Path to the output directory for masks (default: "masks").
    text_prompt (str): Text prompt for LangSAM (default: "tree").
    box_threshold (float): Box threshold for LangSAM (default: 0.24).
    text_threshold (float): Text threshold for LangSAM (default: 0.24).
    """
    # Instantiate LangSAM once
    sam = LangSAM()

    for als_file, dop_image in matches.items():
        # Extract file identifier from the DOP image path
        file_identifier = os.path.splitext(os.path.basename(als_file))[0]

        # Path to the tiles directory for this DOP image
        tile_path = os.path.join(tile_base_dir, file_identifier)

        # Output directory for masks of this tile group
        mask_dir = os.path.join(mask_out_dir, file_identifier)

        # Apply LangSAM to the tiles
        sam.predict_batch(
            images=tile_path,
            out_dir=mask_dir,
            text_prompt=text_prompt,
            box_threshold=box_threshold,
            text_threshold=text_threshold,
            mask_multiplier=255,
            dtype="uint8",
            merge=True,
            verbose=True,
        )

        print(f"Masks created for tiles in {tile_path}. Saved in {mask_dir}.")

In [4]:
from get_file_matches import get_als_tif_matches

In [5]:
# Root directory containing all subfolders
root_dir = "data/Tschernitz"
als_subfolder="als"
tif_subfolder="output"
# Match ALS files with TIF files that contain "merged"
als_tif_matches = get_als_tif_matches(root_dir,als_subfolder=als_subfolder, tif_subfolder=tif_subfolder, contains='final_input')

📂 Extracted identifier '33470-5714' from folder: als_33470-5714
✅ ALS File: data\Tschernitz\als\als_33470-5714\als_33470-5714.laz -> Matched TIF: data\Tschernitz\output\als_33470-5714\als_33470-5714_final_input.tif

📂 Extracted identifier '33470-5715' from folder: als_33470-5715
✅ ALS File: data\Tschernitz\als\als_33470-5715\als_33470-5715.laz -> Matched TIF: data\Tschernitz\output\als_33470-5715\als_33470-5715_final_input.tif

📂 Extracted identifier '33470-5716' from folder: als_33470-5716
✅ ALS File: data\Tschernitz\als\als_33470-5716\als_33470-5716.laz -> Matched TIF: data\Tschernitz\output\als_33470-5716\als_33470-5716_final_input.tif

📂 Extracted identifier '33471-5713' from folder: als_33471-5713
✅ ALS File: data\Tschernitz\als\als_33471-5713\als_33471-5713.laz -> Matched TIF: data\Tschernitz\output\als_33471-5713\als_33471-5713_final_input.tif

📂 Extracted identifier '33471-5714' from folder: als_33471-5714
✅ ALS File: data\Tschernitz\als\als_33471-5714\als_33471-5714.laz -> Mat

In [6]:
# Define output directory for tiles
tile_output_directory = "data/Tschernitz/tiles/dop"

# Create tiles using the matches dictionary
create_tiles_from_matches(als_tif_matches, tile_out_dir=tile_output_directory,  tile_size=(1000, 1000), overlap=0)

Tiles created for als_33470-5714. Saved in data/Tschernitz/tiles/dop\als_33470-5714.
Tiles created for als_33470-5715. Saved in data/Tschernitz/tiles/dop\als_33470-5715.
Tiles created for als_33470-5716. Saved in data/Tschernitz/tiles/dop\als_33470-5716.
Tiles created for als_33471-5713. Saved in data/Tschernitz/tiles/dop\als_33471-5713.
Tiles created for als_33471-5714. Saved in data/Tschernitz/tiles/dop\als_33471-5714.
Tiles created for als_33471-5715. Saved in data/Tschernitz/tiles/dop\als_33471-5715.
Tiles created for als_33471-5716. Saved in data/Tschernitz/tiles/dop\als_33471-5716.
Tiles created for als_33472-5713. Saved in data/Tschernitz/tiles/dop\als_33472-5713.
Tiles created for als_33472-5714. Saved in data/Tschernitz/tiles/dop\als_33472-5714.
Tiles created for als_33472-5715. Saved in data/Tschernitz/tiles/dop\als_33472-5715.
Tiles created for als_33472-5716. Saved in data/Tschernitz/tiles/dop\als_33472-5716.
Tiles created for als_33473-5713. Saved in data/Tschernitz/tiles/

In [7]:
text_prompt="tree"

# Define output directory for masks
mask_output_directory = f"data/Tschernitz/ground_truth_masks/{text_prompt}_masks"

In [ ]:
# Get the first element from the dictionary
first_element = list(als_tif_matches.items())[0]

# Convert the first element to another dictionary
first_match_dict = {first_element[0]: first_element[1]}

In [8]:
# Apply SAM using the matches dictionary
apply_sam_to_tiles_from_matches(als_tif_matches, tile_base_dir=tile_output_directory, mask_out_dir=mask_output_directory,box_threshold=0.24, text_threshold=0.24)

final text_encoder_type: bert-base-uncased
Processing image 01 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_0_0.tif...
Processing image 02 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_0_1.tif...
Processing image 03 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_0_2.tif...
Processing image 04 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_0_3.tif...
Processing image 05 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_0_4.tif...
Processing image 06 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_1_0.tif...
Processing image 07 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_1_1.tif...
Processing image 08 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_1_2.tif...
Processing image 09 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_1_3.tif...
Processing image 10 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_1_4.tif...
Processing image 11 of 25: data/Tschernitz/tiles/dop\als_33470-5714\tile_2_0.tif...
Processing image 12 of 25: data/T

In [9]:
def visualize_overlays(matches, max_images=3):
    """
    Visualize segmentation overlays using Leafmap for the first N images.

    Parameters:
    matches (dict): A dictionary with ALS file paths as keys and corresponding TIF image paths as values.
    max_images (int): Maximum number of TIF images to visualize.
    """

    # Create a Leafmap interactive map
    m = leafmap.Map(center=[-22.1278, -51.4430], zoom=17, height="800px")

    # Add the Satellite Basemap
    m.add_basemap("SATELLITE")

    count = 0  # Counter to control the maximum number of visualizations
    for als_file, tif_file in matches.items():
        # Extract file identifier from the ALS file path
        file_identifier = os.path.splitext(os.path.basename(als_file))[0]

        # Check if the TIF file exists
        if not os.path.exists(tif_file):
            print(f"TIF file not found for {file_identifier}. Skipping visualization.")
            continue

        # Add the TIF file as a raster layer
        try:
            m.add_raster(tif_file, layer_name=f"Segmentation Mask ({file_identifier})", opacity=0.6)
        except Exception as e:
            print(f"Error adding TIF file to map for {file_identifier}: {e}")
            continue

        print(f"Added segmentation mask for {file_identifier} to the map.")

        # Break after visualizing the maximum number of images
        count += 1
        if count >= max_images:
            break

    print(f"Displaying map with {count} segmentation overlays.")
    return m  # Return the map to visualize the images


In [ ]:
# Visualize TIF files from matches
segmentation_map = visualize_overlays(als_tif_matches, max_images=3)

# To display the map
segmentation_map
